<a href="https://colab.research.google.com/github/NathanAugusth/Algoritmos-e-Complexidade/blob/main/Bot_de_Login.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.15.0
    Uninstalling typing_extensions-4.15.0:
      Successfully uninstalled typing_extensions-4.15.0


In [2]:
import json
import os
import time
import tempfile
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException

COOKIES_FILE = 'cookies.json'

def load_cookies(driver):
    """Carrega cookies salvos, se existirem."""
    if os.path.exists(COOKIES_FILE):
        with open(COOKIES_FILE, 'r') as f:
            cookies = json.load(f)
        for cookie in cookies:
            driver.add_cookie(cookie)
        print("Cookies carregados com sucesso.")
    else:
        print("Arquivo de cookies não encontrado.")

def save_cookies(driver):
    """Salva os cookies atuais em um arquivo."""
    cookies = driver.get_cookies()
    with open(COOKIES_FILE, 'w') as f:
        json.dump(cookies, f)
    print("Cookies salvos com sucesso.")

def take_screenshot(driver, filename='error_screenshot.png'):
    """Captura uma screenshot e salva em arquivo."""
    driver.save_screenshot(filename)
    print(f"Screenshot salva em: {filename}")

def login(driver, username, password):
    """Realiza o login no site."""
    try:
        driver.get("https://the-internet.herokuapp.com/login")

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username")))

        driver.find_element(By.ID, "username").send_keys(username)
        driver.find_element(By.ID, "password").send_keys(password)

        driver.find_element(By.CSS_SELECTOR, "button[type='submit']").click()

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "flash")))
        flash_message = driver.find_element(By.ID, "flash").text

        if "You logged into a secure area!" not in flash_message:
            raise ValueError("Login inválido.")

        print("Login realizado com sucesso.")
        return True

    except (NoSuchElementException, TimeoutException) as e:
        print(f"Erro ao encontrar elementos durante o login: {e}")
        take_screenshot(driver)
        return False
    except ValueError as e:
        print(f"Erro de validação: {e}")
        take_screenshot(driver)
        return False
    except WebDriverException as e:
        print(f"Erro no WebDriver: {e}")
        take_screenshot(driver)
        return False

def navigate_and_interact(driver):
    """Navega para a área autenticada e interage (exemplo: extrai informação e preenche um formulário hipotético)."""
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "subheader")))

        extracted_text = driver.find_element(By.CLASS_NAME, "subheader").text
        print(f"Informação extraída: {extracted_text}")

        driver.get("https://the-internet.herokuapp.com/form_authentication")
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "username")))
        driver.find_element(By.ID, "username").send_keys("Dados de teste")
        driver.find_element(By.CSS_SELECTOR, "button[type='submit']").click()

        print("Formulário preenchido e submetido com sucesso.")

    except (NoSuchElementException, TimeoutException) as e:
        print(f"Erro ao encontrar elementos durante a navegação/interação: {e}")
        take_screenshot(driver)
    except WebDriverException as e:
        print(f"Erro no WebDriver durante a navegação: {e}")
        take_screenshot(driver)

def main():
    user_data_dir = tempfile.mkdtemp()

    service = Service()
    options = webdriver.ChromeOptions()
    options.add_argument(f"--user-data-dir={user_data_dir}")
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = None
    try:
        driver = webdriver.Chrome(service=service, options=options)

        driver.get("https://the-internet.herokuapp.com/")
        load_cookies(driver)
        driver.get("https://the-internet.herokuapp.com/secure")

        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "flash")))
            if "You logged into a secure area!" in driver.find_element(By.ID, "flash").text:
                print("Sessão persistida via cookies.")
            else:
                raise TimeoutException
        except TimeoutException:
            if not login(driver, "tomsmith", "SuperSecretPassword!"):
                return
            save_cookies(driver)

        navigate_and_interact(driver)

    except WebDriverException as e:
        print(f"Erro ao iniciar o WebDriver: {e}")
        if driver:
            take_screenshot(driver)

    finally:
        if driver:
            driver.quit()
        if os.path.exists(user_data_dir):
            import shutil
            shutil.rmtree(user_data_dir)

if __name__ == "__main__":
    main()

Arquivo de cookies não encontrado.
Login realizado com sucesso.
Cookies salvos com sucesso.
Informação extraída: Welcome to the Secure Area. When you are done click logout below.
Erro ao encontrar elementos durante a navegação/interação: Message: 
Stacktrace:
#0 0x576965b56cba <unknown>
#1 0x5769655cb8e0 <unknown>
#2 0x57696561d580 <unknown>
#3 0x57696561d771 <unknown>
#4 0x57696566b864 <unknown>
#5 0x57696564309d <unknown>
#6 0x576965668c63 <unknown>
#7 0x576965642e43 <unknown>
#8 0x57696560f9a8 <unknown>
#9 0x576965610621 <unknown>
#10 0x576965b1a8b8 <unknown>
#11 0x576965b1e5e2 <unknown>
#12 0x576965b01683 <unknown>
#13 0x576965b1f135 <unknown>
#14 0x576965ae680f <unknown>
#15 0x576965b434c8 <unknown>
#16 0x576965b436a2 <unknown>
#17 0x576965b55dc3 <unknown>
#18 0x7d6818094ac3 <unknown>

Screenshot salva em: error_screenshot.png
